In [55]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [56]:
batch_size = 32
img_height = 16
img_width = 8

train_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset",
    validation_split=0.3,
    subset="training",
    seed=256,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset",
    validation_split=0.3,
    subset="validation",
    seed=256,
    image_size=(img_height, img_width),
    batch_size=batch_size)
    
normalization_layer = tf.keras.layers.Rescaling(1./255)

Found 6598 files belonging to 42 classes.
Using 4619 files for training.
Found 6598 files belonging to 42 classes.
Using 1979 files for validation.


In [57]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

0.1764706 0.9843138


In [58]:
num_classes = len(os.listdir("Dataset"))

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(75, activation='relu'),
  # tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])

In [59]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=5*1e-6)
# model.compile(optimizer=opt, 
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

from tensorflow.keras.callbacks import TensorBoard
import random
NAME = "basic"+str(random.randint(1,1000))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
# cd "C:\src\Python\Smart Clicker\V9\"
# tensorboard --logdir="logs/"

C:\Users\Admin\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [60]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50, callbacks=[tensorboard]
)


Epoch 1/50
145/145 [==============================] - 8s 51ms/step - loss: 2.6297 - accuracy: 0.4025 - val_loss: 1.4022 - val_accuracy: 0.6387
Epoch 2/50
145/145 [==============================] - 0s 3ms/step - loss: 0.9346 - accuracy: 0.7623 - val_loss: 0.6886 - val_accuracy: 0.8044
Epoch 3/50
145/145 [==============================] - 0s 2ms/step - loss: 0.5324 - accuracy: 0.8461 - val_loss: 0.4370 - val_accuracy: 0.8919
Epoch 4/50
145/145 [==============================] - 0s 3ms/step - loss: 0.3693 - accuracy: 0.8950 - val_loss: 0.3139 - val_accuracy: 0.9040
Epoch 5/50
145/145 [==============================] - 0s 3ms/step - loss: 0.2773 - accuracy: 0.9253 - val_loss: 0.2457 - val_accuracy: 0.9050
Epoch 6/50
145/145 [==============================] - 0s 3ms/step - loss: 0.2120 - accuracy: 0.9465 - val_loss: 0.1934 - val_accuracy: 0.9232
Epoch 7/50
145/145 [==============================] - 1s 4ms/step - loss: 0.1675 - accuracy: 0.9584 - val_loss: 0.1521 - val_accuracy: 0.9409
Epoch

In [ ]:
model_name = "item_reader_seq.model"
model.save(model_name)

In [66]:
import numpy as np
import cv2
X=[]
y=[]

for index, batch in enumerate(val_ds):
    X, y = batch
    X = X.numpy()
    y = y.numpy()
    
    y_predict = model.predict(X)
    y_predict = np.argmax(y_predict, axis=1)
    # for i in range(32):
    #     if y[i]!=y_predict[i]:
    #         print(index, i)
    #         cv2.imwrite(f"{random.randint(1, 100)}.png", X[i])
    
    cv2.imshow('image', X[0]) 
    cv2.imwrite("image.png", cv2.cvtColor(X[0], cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    print(y[0], y_predict[0])
    # print(y_predict)
    # break



37 37
19 19
20 20
39 39
26 26
11 11
23 23
28 28
11 11
3 3
31 31
1 1
40 40
33 33
32 32
39 39
23 23
27 27
9 9
21 21
39 39
21 21
31 31
19 19
40 40
17 17


KeyboardInterrupt: 

In [69]:
# Need Bandicam to take a screenshot when Minecraft in full screen mode
Z = [(4, 4), (4, 76), (4, 148), (4, 220), (4, 292),
    (4, 364), (4, 436), (4, 508), (4, 580), (76, 4),
    (76, 76), (76, 148), (76, 220), (76, 292), (76, 364),
    (76, 436), (76, 508), (76, 580), (148, 4), (148, 76),
    (148, 148), (148, 220), (148, 292), (148, 364), (148, 436),
    (148, 508), (148, 580), (236, 4), (236, 76), (236, 148),
    (236, 220), (236, 292), (236, 364), (236, 436), (236, 508),
    (236, 580)]
from tensorflow.keras.models import load_model
model = load_model("item_reader_seq.model")
import numpy as np
import cv2
import os
img = cv2.imread("D:/Bi/Record/javaw 2021-11-05 21-41-48-099.jpg")
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
img = img[540:845, 636:1285]
# img = img[480:845, 600:1285]

Slot = []
for x, y in Z:
    Slot.append(img[x:x+64, y:y+32])
    # Slot.append(img)

for n in range(36):
    Slot[n] = cv2.resize(Slot[n], (32//4, 64//4), interpolation=cv2.INTER_AREA)

# model([Slot,], training=False)
# x=np.array(Slot)
x=model.predict(np.array(Slot))
# .numpy()
x = np.argmax(x, axis=1)

x

array([29, 27, 27, 11, 27, 27, 27, 27, 27,  7, 27, 27, 27, 27, 27, 27, 27,
       27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 34,  8, 10,  9, 13, 14,
       15, 16], dtype=int64)